In [66]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [67]:
%%writefile wyndor.mod
set VEICULOS;

param capacidade_pessoas {VEICULOS};
param capacidade_bagagem {VEICULOS};
param custo {VEICULOS};
param tempo_viagem {VEICULOS};
param unidades_disponiveis {VEICULOS};

param total_pessoas := 17750;
param tempo_maximo := 600;

var Viagens{VEICULOS} >= 0, integer;

# Função Objetivo: Minimizar o custo total
minimize Custo_Total:
    sum {v in VEICULOS} custo[v] * Viagens[v];

# Restricao: Capacidade de transporte de pessoas
subject to Capacidade_Pessoas:
    sum {v in VEICULOS} capacidade_pessoas[v] * Viagens[v] >= total_pessoas;

# Restricao: Capacidade de transporte de bagagem
subject to Capacidade_Bagagem:
    sum {v in VEICULOS} capacidade_bagagem[v] * Viagens[v] >= 177500;

# Restricao: Tempo total de viagem
subject to Tempo_Total:
    sum {v in VEICULOS} tempo_viagem[v] * Viagens[v] <= tempo_maximo;

# Restricao: Limite de unidades disponíveis
subject to Limite_Unidades {v in VEICULOS}:
    Viagens[v] <= unidades_disponiveis[v];

Overwriting wyndor.mod


In [68]:
%%writefile wyndor.dat
data;

set VEICULOS := "6_Ton" "1_4_Ton" "Helicoptero" "Onibus" "Micro_onibus" "Carro";

param capacidade_pessoas :=
"6_Ton"      20
"1_4_Ton"     5
"Helicoptero" 10
"Onibus"      30
"Micro_onibus" 15
"Carro"        5;

param capacidade_bagagem :=
"6_Ton"      1000
"1_4_Ton"     20
"Helicoptero" 50
"Onibus"      1000
"Micro_onibus" 500
"Carro"        100;

param custo :=
"6_Ton"      10
"1_4_Ton"     4
"Helicoptero" 75
"Onibus"      5
"Micro_onibus" 3
"Carro"        2;

param tempo_viagem :=
"6_Ton"      60
"1_4_Ton"     45
"Helicoptero" 10
"Onibus"      45
"Micro_onibus" 30
"Carro"        30;

param unidades_disponiveis :=
"6_Ton"      10
"1_4_Ton"     20
"Helicoptero" 15
"Onibus"      10
"Micro_onibus" 5
"Carro"        60;

end;



Overwriting wyndor.dat


[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)

In [69]:
%%ampl_eval
model wyndor.mod;
data wyndor.dat;
option solver cbc;
solve;


	presolve: constraint Capacidade_Bagagem cannot hold:
		body >= 177500 cannot be <= 25510; difference = 151990
	presolve: constraint Capacidade_Pessoas cannot hold:
		body >= 17750 cannot be <= 890; difference = 16860
